# Setup

## Download Images from Google Drive

[Google Drive Link](https://drive.google.com/file/d/1Xi5qlD4vJkN5YDcvqMXST2jZXVSh7VWm/view?usp=sharing)

## Extract Zip File

In [ ]:
import zipfile

# Make sure the zip file is uploaded to Colab, or in same directory as this file
path_to_zip_file = "./images.zip"
directory_to_extract_to = "./"
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)


In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(img_path):
    image = mpimg.imread(img_path)
    plt.imshow(image)
    plt.show()


current_dir = os.path.abspath(".")

test_image_path = os.path.join(current_dir, 'images', 'train', 'dog', 'dog.1.jpg')
show_image(test_image_path)

# Importing Data from directories using Tensorflow

In [ ]:
import tensorflow as tf

data_dir = os.path.join(current_dir, 'images', 'train')

img_height = 128
img_width = 128
batch_size = 15
random_seed = 32601

# where I got code from https://www.tensorflow.org/tutorials/load_data/images
dataset = tf.keras.utils.image_dataset_from_directory(  # https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
    data_dir,
    validation_split=0.25,
    subset="training",
    seed=random_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    interpolation='bilinear')

dataset.class_names


In [ ]:
# imported again
import matplotlib.pyplot as plt # https: // matplotlib.org/stable/gallery/index

# what are the class names?
class_names = dataset.class_names
print(class_names)

plt.figure(figsize=(10, 10))
for images, labels in dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


# They are stretched...

**Does that matter?**

Idk, does it??

**How can I change it?**

What are some ideas?

crop_to_aspect_ratio (change it and show)

In [ ]:
train = tf.keras.utils.image_dataset_from_directory(  # https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
    data_dir,
    validation_split=0.25,
    subset="training",
    seed=random_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    interpolation='bilinear',
    label_mode='categorical')

valid = tf.keras.utils.image_dataset_from_directory(  # https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory
    data_dir,
    validation_split=0.25,
    subset="validation",
    seed=random_seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    interpolation='bilinear',
    label_mode='categorical',
    )


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

# datagen = ImageDataGenerator(  # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
#     samplewise_center=True,
#     rotation_range=15,
#     zoom_range=0.1,
#     width_shift_range=0.1,
#     height_shift_range=0.1, 
#     horizontal_flip=True,
#     vertical_flip=False,
# )

# datagen.flow_from_dataframe(  # https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
#     data_dir, 
#     target_size=(img_height, img_width), 
#     batch_size=batch_size, 
#     class_mode='categorical')


# Create Model

 - Input Layer: It represent input image data. It will reshape image into single diminsion array. Example your image is 64x64 = 4096, it will convert to (4096,1) array.
 - Conv Layer: This layer will extract features from image.
 - Pooling Layer: This layerreduce the spatial volume of input image after convolution.
 - Fully Connected Layer: It connect the network from a layer to another layer
 - Output Layer: It is the predicted values layer.

In [ ]:
from tensorflow import keras # https://www.tensorflow.org/api_docs/python/tf/keras/
from tensorflow.keras.models import Sequential # https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
from tensorflow.keras.layers import ( # https://www.tensorflow.org/api_docs/python/tf/keras/layers
    Dense,              # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
    Conv2D,             # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
    MaxPool2D,          # https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
    Flatten,            # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Flatten
    Dropout,            # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
    BatchNormalization, # https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
)

# Sequential model from https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification/notebook
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
# 2 because we have cat and dog classes
model.add(Dense(2, activation='softmax'))


In [ ]:
print(model.summary())

# Compile and Train Model

In [ ]:
from tensorflow import keras

model.compile(loss=tf.losses.CategoricalCrossentropy(), optimizer='rmsprop', metrics=["accuracy"])


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau # https://keras.io/callbacks/

earlystop = EarlyStopping(patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=2,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)
callbacks = [earlystop, learning_rate_reduction]


In [ ]:
total_validate = 18750
total_train = 6250

print ("Total validation images: ", total_validate)
print ("Total train images: ", total_train)

print("Validation Steps: ", total_validate//batch_size)
print("Train Steps: ", total_train//batch_size)

history = model.fit(train, validation_data=valid, epochs=50, callbacks=callbacks,
                    validation_steps=total_validate//batch_size, steps_per_epoch=total_train//batch_size)


# How did we do?

In [ ]:
import numpy as np

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 128, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['acc'], color='b', label="Training accuracy")
ax2.plot(history.history['val_acc'], color='r', label="Validation accuracy")
ax2.set_xticks(np.arange(1, 128, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()
